In [ ]:
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors

#importing the Linear Discriminant Analysis function from the Scikit-learn library
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing the Quadratic Discriminant Analysis function from the Scikit-learn library.
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# #############################################################################
# Colormap- essentially a mapping from numbers to colors
#  the red component starts at full intensity and gradually decreases to 70% intensity
# as it moves from the lower to the upper end of the scale. Note: this is not about location.
# The same pattern is followed for the green and blue components.
cmap = colors.LinearSegmentedColormap(
    "red_blue_classes",
    {
        "red": [(0, 1, 1), (1, 0.7, 0.7)],
        "green": [(0, 0.7, 0.7), (1, 0.7, 0.7)],
        "blue": [(0, 0.7, 0.7), (1, 1, 1)],
    },
)
# Registering the new colormap with Matplotlib (which will be imported as plt),
# so that it can be used later on in the script for creating plots
#plt.cm.register_cmap(cmap=cmap) # You should disable this line if you want to run this code again


# #############################################################################
# Dataset 1: Generate datasets with 600 samples (300 for each class) and 2 features.
# The samples are drawn from a 2-dimensional Gaussian distribution.
# The labels for the first 300 samples are all zeros, and
# the labels for the second 300 samples are all ones.
# Note that the we have the same C (scale of the randomness) for class 1 and class 0
def dataset_fixed_cov():
    """Generate 2 Gaussians samples with the same covariance matrix"""
    n, dim = 300, 2
    np.random.seed(0)
    C = np.array([[0.0, -0.23], [0.83, 0.23]])
    X = np.r_[ # stacking rows by rows
        np.dot(np.random.randn(n, dim), C), # input data X for class 1
        np.dot(np.random.randn(n, dim), C) + np.array([1, 1]),
    ]
    y = np.hstack((np.zeros(n), np.ones(n))) # labels 0 for one class and 1 for the other class
    return X, y

# Dataset 2: Generate another 600 samples. The key difference is that C is not the same for class 0 and class 1
def dataset_cov():
    """Generate 2 Gaussians samples with different covariance matrices"""
    n, dim = 300, 2
    np.random.seed(0)
    C = np.array([[0.0, -1.0], [2.5, 0.7]]) * 2.0 # Create a 2X2 matrix scaled by 2
    X = np.r_[
        np.dot(np.random.randn(n, dim), C),
        np.dot(np.random.randn(n, dim), C.T) + np.array([1, 4]),
    ]
    y = np.hstack((np.zeros(n), np.ones(n))) # 0, 0, 0, .... 1, 1, 1 vector
    return X, y


# #############################################################################
# Plot functions
def plot_data(classifier, X, y, y_pred, fig_index):
    splot = plt.subplot(2, 2, fig_index)
    if fig_index == 1:
        plt.title("Linear Discriminant Analysis")
        plt.ylabel("Data with\n fixed covariance")
    elif fig_index == 2:
        plt.title("Quadratic Discriminant Analysis")
    elif fig_index == 3:
        plt.ylabel("Data with\n varying covariances")

    tp = y == y_pred  # True Positive
    tp0, tp1 = tp[y == 0], tp[y == 1]
    X0, X1 = X[y == 0], X[y == 1]
    X0_tp, X0_fp = X0[tp0], X0[~tp0]
    X1_tp, X1_fp = X1[tp1], X1[~tp1]

    # class 0: dots
    plt.scatter(X0_tp[:, 0], X0_tp[:, 1], marker=".", color="red")
    plt.scatter(X0_fp[:, 0], X0_fp[:, 1], marker="x", s=20, color="#990000")  # dark red

    # class 1: dots
    plt.scatter(X1_tp[:, 0], X1_tp[:, 1], marker=".", color="blue")
    plt.scatter(
        X1_fp[:, 0], X1_fp[:, 1], marker="x", s=20, color="#000099"
    )  # dark blue

    # class 0 and 1 : areas
    nx, ny = 200, 100
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx), np.linspace(y_min, y_max, ny))
    Z = classifier.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)
    plt.pcolormesh(
        xx, yy, Z, cmap="red_blue_classes", norm=colors.Normalize(0.0, 1.0), zorder=0
    )
    plt.contour(xx, yy, Z, [0.5], linewidths=2.0, colors="white")

    # means
    plt.plot(
        classifier.means_[0][0],
        classifier.means_[0][1],
        "*",
        color="yellow",
        markersize=15,
        markeredgecolor="grey",
    )
    plt.plot(
        classifier.means_[1][0],
        classifier.means_[1][1],
        "*",
        color="yellow",
        markersize=15,
        markeredgecolor="grey",
    )

    return splot


def plot_ellipse(splot, mean, cov, color):
    v, w = linalg.eigh(cov)
    u = w[0] / linalg.norm(w[0])
    angle = np.arctan(u[1] / u[0])
    angle = 180 * angle / np.pi  # convert to degrees
    # filled Gaussian at 2 standard deviation
    ell = mpl.patches.Ellipse(
        mean,
        2 * v[0] ** 0.5,
        2 * v[1] ** 0.5,
        180 + angle,
        facecolor=color,
        edgecolor="black",
        linewidth=2,
    )
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(0.2)
    splot.add_artist(ell)
    splot.set_xticks(())
    splot.set_yticks(())


def plot_lda_cov(classifier, splot): #plotting the covariance ellipses
    plot_ellipse(splot, classifier.means_[0], classifier.covariance_, "red")
    plot_ellipse(splot, classifier.means_[1], classifier.covariance_, "blue")


def plot_qda_cov(classifier, splot):
    plot_ellipse(splot, classifier.means_[0], classifier.covariance_[0], "red")
    plot_ellipse(splot, classifier.means_[1], classifier.covariance_[1], "blue")


plt.figure(figsize=(10, 8), facecolor="white")
plt.suptitle(
    "Linear Discriminant Analysis vs Quadratic Discriminant Analysis",
    y=0.98,
    fontsize=15,
)
# i - index starting from 0
# enumerate (X,y) from all data generated. When i = 0, (X,y) come from dataset_fixed_cv. when i=1, from dataset_cov()
for i, (X, y) in enumerate([dataset_fixed_cov(), dataset_cov()]):
    print('i=',i, '  (X,y): ', (X.shape, y.shape))
    # Linear Discriminant Analysis
    # first estimating the mean vectors and the covariance matrices for each class from the training dataset. Saved in lda
    # Perform Fisher's Discriminant analysis considering maximizing between-class difference and minimizing within class variance
    classifier = # You should let classifier store covariance by setting store_covariance=True. Otherwise, you will have trouble with plot_lda_cov()
    y_pred =
    splot = plot_data(, fig_index=2 * i + 1) # Use the plot function defined above
    plot_lda_cov(classifier, splot) #plotting the covariance ellipses, function defined above
    plt.axis("tight") #djusts the axes of the plot such that all the data is visible and the plot area is maximized

    # Quadratic Discriminant Analysis
    classifier =  # You should let classifier store covariance by setting store_covariance=True. Otherwise, you will have trouble with plot_qda_cov()
    y_pred =
    splot = plot_data(, fig_index=2 * i + 2)
    plot_qda_cov(classifier, splot)
    plt.axis("tight")
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()